# Setup Workspace

In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
RANDOM_SEED = 420
dataset_file = 'formated_dataframe.csv'

In [3]:
torch.manual_seed(RANDOM_SEED)

# Model Selection [Advanced Pre-Trained Model]

In [4]:
df = None
try:
    %store -r df
except KeyError:
    df = pd.read_csv('formated_dataframe.csv')

In [5]:
train_df, test_df = train_test_split(df, test_size=0.2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
class SentencePairDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        sentence1 = str(self.data.sentence1[index])
        sentence2 = str(self.data.sentence2[index])
        label = int(self.data.label[index])
        inputs = self.tokenizer.encode_plus(
            sentence1,
            sentence2,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [7]:
train_dataset = SentencePairDataset(train_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = SentencePairDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased').to(device)
device.type

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

C:\Workspace\PMLDL_Q2C\venv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cla

'cuda'

In [9]:
def train(epoch):
    optimizer = AdamW(model.parameters(), lr=1e-5)

    model.train()
    for _, data in enumerate(train_loader, 0):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        labels = data['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(ids, mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


for epoch in range(3):
    train(epoch)

# Save the model
model_path = 'bert_finetuned.pth'
torch.save(model.state_dict(), model_path)

C:\Workspace\PMLDL_Q2C\venv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


AttributeError: 'DataFrame' object has no attribute 'sentence1'

In [ ]:
def get_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

In [ ]:
def find_most_similar(query, dataframe):
    query_embedding = get_embedding(query)
    max_similarity = -1
    most_similar_sentence = ""

    for sentence in dataframe['sentence']:
        sentence_embedding = get_embedding(sentence)
        similarity = cosine_similarity(query_embedding, sentence_embedding)

        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_sentence = sentence

    return most_similar_sentence, max_similarity


In [ ]:
query = "Do you want to go to a bar?"
most_similar_sentence, similarity_score = find_most_similar(query, df)
print("Most Similar Sentence:", most_similar_sentence)
print("Similarity Score:", similarity_score)